In [1]:
# Used for indices based on the indice list
def dropdown_options(listname):
	dropdown_list = []
	for item in listname:
		dropdown_list.append({'label':str(item), 'value':item})
	return dropdown_list

# Used to convert the symbol of index to the name of the index
def get_index_symbol(index_list):
    new_list = []
    for i in range(len(index_list)):
        new_list.append(list(indices[indices['Name'] == index_list[i]]['Symbol'])[0])
    return new_list

# Convert dataset from json back to pandas
# Function should be used for dcc.store as all dcc.store variables must be converted to json
def data_to_pandas(dataset):
    json_dataframe = json.loads(dataset)
    json_dataframe = pd.read_json(json.dumps(json_dataframe))
    return json_dataframe

# Prepares the dataset for the correlation graph
# Function should be used to take in dcc.store variable and covert that from json and create the df
def correlation_data(func_data, ticker_input_func, radio_input):
    func_data = data_to_pandas(func_data)
    column_list = []
    for i in range(len(ticker_input_func)):
        column_list.append(ticker_input_func[i])
    df = pd.DataFrame(columns = column_list)
    for ticker in ticker_input_func:
        df[ticker] = func_data[str((radio_input,ticker))]
    df = df.corr()
    return df

# Obtain stock analytics dataset and put it into a dictionary
def obtain_stock_analytics_data(ticker_input):
    ticker_input = ticker_input.split(",")    
    stock_analytics_data = []
    for ticker in ticker_input:
        url = 'https://www.marketwatch.com/investing/stock/'+ticker.lower() + '/company-profile?mod=mw_quote_tab'
        time.sleep(2.5)
        stock_analytics_data.append(pd.read_html(url))
    return stock_analytics_data
        
# convert the stock analytics into a readable table which is then used for graph data
def stock_analytics(summary_data, ticker_input, index):
        # Df 4 for valuation
        # Df 6 for liquidity
        # Df 7 for profitability
        # Df 8 for capitalization
        # Df 10 for insider actions
    ticker_input = ticker_input.split(",")  
    summary_data_dict = {}
    summary_data_dict['Description'] = summary_data[0][index][0]
    for i in range(len(ticker_input)):
        summary_data_dict[ticker_input[i]] = summary_data[i][index][1]
    summary_data_df = pd.DataFrame(summary_data_dict)
    return summary_data_df

# Obtain necessary news articles based on ticker_input, start date and end date
def get_news_articles(ticker_input_func,start_date_func,end_date_func):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    
    ticker_input_func = ticker_input_func.split(',')
    df_news = pd.DataFrame()     
    
    for ticker in ticker_input_func:
        time.sleep(2.5)
        temp_news = pd.DataFrame()
        googlenews=GoogleNews(start= start_date_func,end=end_date_func)
        googlenews.search(yf.Ticker(ticker).info['longName'])
        temp_news = pd.DataFrame(googlenews.result())
        temp_news['Ticker'] = ticker
        df_news = df_news.append(temp_news)
    return df_news

# Cleanup news articles, obtain article title, media, datetime and full text
def news_articles_cleanup(df):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    news_list = []
    df = df.reset_index()
    for ind in df.index:
        try:
            time.sleep(2.5)
            dict_news={}
            article = Article(df['link'][ind],config=config)
            article.download()
            article.parse()
            article.nlp()
            dict_news['Date']=df['date'][ind]
            dict_news['Ticker']=df['Ticker'][ind]
            dict_news['Media']=df['media'][ind]
            dict_news['Title']=article.title
            dict_news['Article']=article.text
            news_list.append(dict_news)
        except:
            pass
    cleaned_df = pd.DataFrame(news_list)
    cleaned_df.replace('',np.nan, inplace=True)
    cleaned_df = cleaned_df.dropna()
    return cleaned_df

# Sentiment analysis of scores based on news_articles_cleanup dataset
def sentiment_scores_df(dataset):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    sentiment_list = []
    sentiment_dict = {}
    sid = SentimentIntensityAnalyzer()
    try: 
        dataset = dataset.reset_index()
    except:
        pass
    # Creates the sentiment dictionary which will then be passed off as a json file
    for i in range(len(dataset['Article'])):
        sentiment_dict = {}
        polarity_scores = sid.polarity_scores(dataset['Article'][i])
        sentiment_dict['Ticker'] = dataset['Ticker'][i]
        #sentiment_dict['Article'] = dataset['Article'][i]
        sentiment_dict['Date'] = dataset['Date'][i]
        sentiment_dict['Negative'] = polarity_scores['neg']
        sentiment_dict['Positive'] = polarity_scores['pos']
        sentiment_dict['Neutral'] = polarity_scores['neu']
        sentiment_dict['Compound'] = polarity_scores['compound']
        sentiment_list.append(sentiment_dict)
    return pd.DataFrame(sentiment_list)

# Obtains the top 25 cryptocurrency data from yahoofinance 
def top25crypto():
    url = requests.get('https://finance.yahoo.com/cryptocurrencies').content
    top25_table = pd.read_html(url)[0]
    top25_table.drop(['52 Week Range','1 Day Chart'], axis = 1, inplace = True)
    columns = ['% Change','Market Cap','Volume in Currency (Since 0:00 UTC)','Volume in Currency (24Hr)','Total Volume All Currencies (24Hr)','Circulating Supply']
    for j in range(len(columns)):
        mylist = []
        for i in range(len(top25_table[columns[j]])):
            x = top25_table[columns[j]][i]
            x = float(''.join(list(x)[0:len(list(x))-1]))
            mylist.append(x)
        top25_table[columns[j]] = mylist
    top25_table['% Change'] = top25_table['% Change'] / 100
    return top25_table

# Use the dictionary for the title
radio_dict = {'Adj Close':'Adjusted Close', 'Open':'Open','Close':'Close','High':'High','Low':'Low','Volume':'Volume'}

# Use the dictionary for the stock analytics
stock_analytics_dict = {4:'Valuation',6:'Liquidity',7:'Profitability',8:'Capitalization'}

# Top 5 crypto symbols used to collect data
crypto_symbols = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'SOL-USD']

In [ ]:
# Import necessary libraries
# Impot dash related libraries
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
# Visualization libraries
import plotly.graph_objs as go
import seaborn as sns
import plotly.express as px
# Import pandas related libraries
import pandas as pd
import pandas_datareader.data as web
import numpy as np
# Import news articles libraries
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
import yfinance as yf
# Import sentiment analyzer libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Import other libraries
import datetime as dt
import json
import time
import requests

# External stylesheets for the application
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Setup dash application
app = dash.Dash(external_stylesheets=external_stylesheets)

# Lists used for DCC dropdowns
indices = pd.read_csv('List_of_Indices.csv')

# color used for H1 (Title)
color = {'background':'#111111','text':'#000709'}

# App layout
app.layout = html.Div(children = [
    # H1 Title for the Dashboard
    html.H1('Portfolio Dashboard', style = {'textAlign':'center', 'color':color['text'], 'size':20, 'font-weight':'bold'}),
    
# ------------------------------------------ PART 1 - INDEX UI ---------------------------------------
    
    dcc.Tabs([
        dcc.Tab(label='Index Analysis', children=[ 
    
            # H2 Title for the dashboard
            html.H2('Index Analysis', style = {'textAlign':'left', 'color':color['text'], 'size':14, 'font-weight':'bold'}),
            # Div used to have 2 labels
            html.Div(className = 'row', style = {'display':'flex'}, children = [
                html.Label('Please select the indices:', style = {'textAlign':'left', 'font-weight':'bold'}),
                html.Label('Please select the appropriate dates:', style = {'margin':'auto', 'font-weight':'bold'})      
            ]),
            html.Div(className = 'row', style = {'display':'flex'},children = [
                dcc.Dropdown(id = 'indice_selector', options = dropdown_options(list(indices['Name'])), value = indices['Name'][0:2],
                        multi = True, style = {'font-size': '14px','width':'400px'}),
                dcc.DatePickerRange(id = 'indice_date_selector',start_date_placeholder_text="Start Period", end_date_placeholder_text="End Period",
                            style={'font-size': '14px','width':'400px', 'margin':'auto'})
            ]),
            html.Label('Please select the type of graph:', style = {'textAlign':'left', 'font-weight':'bold'}),
            # Radio Items used to update the index graph
            dcc.RadioItems(id = 'index_radio_items',options = [
                                                    {'label':'Adjusted Close', 'value':'Adj Close'},
                                                    {'label':'Open','value':'Open'},
                                                    {'label':'Close','value':'Close'},
                                                    {'label':'High','value':'High'},
                                                    {'label':'Low','value':'Low'},
                                                    {'label':'Volume','value':'Volume'}],
                           value = 'Volume',
                           labelStyle={'display': 'inline-block'}),
            # Index Graph - shows line grpah based on inputs from user
            dcc.Graph(id = 'indice_graph', style ={'height':'680px'}),
            # Correlation Graph shows correlation between indexs based on inputs from user
            dcc.Graph(id = 'indice_correlation'),
            # Store value for the indice graph and indice correlation plot
            dcc.Store(id='indice_graph_data')
    
    ]),     
            
# ------------------------------------------ PART 2 - STOCK PRICE MOVEMENT UI ---------------------------------------
    
    dcc.Tab(label='Stock Price Movement', children=[
        # Stock portion of the application
        html.H2('Stock Analysis', style = {'textAlign':'left', 'color':color['text'], 'font-weight':'bold'}),
        # Div used to format the labels for ticker and dates
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            html.Label('Please input the tickers and click submit:', style = {'textAlign':'left', 'font-weight': 'bold'}),
            html.Label('Please select the appropriate dates:', style = {'margin':'auto', 'font-weight': 'bold'})
        ]),
        # Div used to format the inputs for stock graph and stock correlation graph
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            dcc.Input(id = 'ticker_input', type = 'text', style = {'width':'500px', 'font-size':'14px'}),
            dcc.DatePickerRange(id = 'stock_date_selector',start_date_placeholder_text="Start Period", end_date_placeholder_text="End Period",
                style={'font-size': '14px','width':'400px', 'margin':'auto'})
        ]),
         html.Label('Please select the type of graph:', style = {'textAlign':'left', 'font-weight': 'bold'}),
        # Radio Items used to update the stock graph
        dcc.RadioItems(id = 'stock_radio_items',options = [
                                                {'label':'Adjusted Close', 'value':'Adj Close'},
                                                {'label':'Open','value':'Open'},
                                                {'label':'Close','value':'Close'},
                                                {'label':'High','value':'High'},
                                                {'label':'Low','value':'Low'},
                                                {'label':'Volume','value':'Volume'}],
                       value = 'Volume',
                       labelStyle={'display': 'inline-block'}),
        dcc.Store(id = 'stock_graph_data'),
        # Submit button used to update the stock graph
        html.Button('Submit', id = 'stock_button', n_clicks = 0),
        # Stock graph - shows lien graph based on inputs from the user
        dcc.Graph(id = 'stock_graph', style ={'height':'680px'}),
        # Correlation graph - shows correlation based on inputs from the user
        dcc.Graph(id = 'stock_correlation')
    ]),
    
# ------------------------------------------ PART 3 - STOCK FINANCIAL PERFORMANCE UI ---------------------------------------
    
    dcc.Tab(label='Stock Financial Performance', children=[
        
        dcc.Store(id = 'stock_analytics_data'),
        html.H2('Stock Analytics', style = {'textAlign':'left', 'color':color['text'], 'font-weight':'bold'}),
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            html.Label('Stock Valuation (MarketWatch)', style = {'textAlign':'left', 'font-weight': 'bold'}),
            html.Label('Stock Liquidity (MarketWatch)', style = {'margin':'auto', 'font-weight': 'bold'})
        ]),
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            dcc.Graph(id = 'stock_valuation', style = {'width':'900px', 'height':'680px'}),
            dcc.Graph(id = 'stock_liquidity', style={'width':'900px', 'height':'680px', 'margin':'auto'})
        ]),
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            html.Label('Stock Profitability (MarketWatch)', style = {'textAlign':'left', 'font-weight': 'bold'}),
            html.Label('Stock Capitalization (MarketWatch)', style = {'margin':'auto', 'font-weight': 'bold'})
        ]),
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            dcc.Graph(id = 'stock_profitability', style = {'width':'900px', 'height':'680px'}),
            dcc.Graph(id = 'stock_capitalization', style={'width':'900px', 'height':'680px', 'margin':'auto'})
        ])
    ]),
    
        
        
# ------------------------------------------ PART 4 - STOCK SENTIMENT ANALYSIS UI ---------------------------------------
    
    dcc.Tab(label='Stock Sentiment Analysis', children=[
        
        dcc.Store(id = 'articles_data'),
        html.H2('Stock Sentiment Analysis', style = {'textAlign':'left','color':color['text'], 'font-weight':'bold'}),
        html.Label('Please select the appropriate sentiment graph: ', style = {'margin':'auto', 'font-weight': 'bold'}),
        dcc.RadioItems(id = 'sentiment_radio_items',options = [
                                                    {'label':'Positive', 'value':'Positive'},
                                                    {'label':'Negative','value':'Negative'},
                                                    {'label':'Neutral','value':'Neutral'},
                                                    {'label':'Compound','value':'Compound'}],
                      value = 'Compound', labelStyle={'display': 'inline-block'}                  
                      ),
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            dcc.Graph(id = 'line_graph_sentiment_graph', style = {'width':'900px', 'height':'680px'}),
            dcc.Graph(id = 'bar_graph_sentiment_graph', style={'width':'900px', 'height':'680px', 'margin':'auto'})
        ])
    ]),      
        
# ------------------------------------------ PART 5 - CRYPTO ANALYSIS UI ---------------------------------------
    
    dcc.Tab(label='Cryptocurrency Analysis', children=[
        
        # Stock portion of the application
        html.H2('Cryptocurrency Analysis', style = {'textAlign':'left', 'color':color['text'], 'font-weight':'bold'}),
        # Label for type of input
        html.Label('Please input the crypto symbols:', style = {'textAlign':'left', 'font-weight': 'bold'}),
        #Crypto ticker input
        dcc.Input(id = 'crypto_ticker_input', type = 'text', style = {'width':'500px', 'font-size':'14px'}),
        # Div used to format the labels for ticker and dates
        html.Label('Please select the type of graph:', style = {'textAlign':'left', 'font-weight': 'bold'}),  
        # Radio Items used to update the stock graph
        html.Div(className = 'row', style = {'display':'flex'}, children = [
                                dcc.RadioItems(id = 'crypto_radio_items',options = [
                                                                                    {'label':'Adjusted Close', 'value':'Adj Close'},
                                                                                    {'label':'Open','value':'Open'},
                                                                                    {'label':'Close','value':'Close'},
                                                                                    {'label':'High','value':'High'},
                                                                                    {'label':'Low','value':'Low'},
                                                                                    {'label':'Volume','value':'Volume'}],
                                                           value = 'Volume',
                                                           labelStyle={'display': 'inline-block'}),
               ]),
        html.Br(),
        # Refresh button used to update the stock graph
        html.Button('Submit', id = 'crypto_refresh_button', n_clicks = 0, style = {'margin':'auto'}),
        # Gets the prices for the top 5 crypto being traded
        dcc.Store(id = 'top5crypto_data'),
        # Gets market capitalization of top 25 crypto graphs
        dcc.Store(id = 'top25graph_data'),
        # Stock graph - shows lien graph based on inputs from the user
        dcc.Graph(id = 'crypto_linegraph', style = {'height':'800px'}),
        # Correlation graph - shows correlation based on inputs from the user
        html.Div(className = 'row', style = {'display':'flex'}, children = [
            dcc.Graph(id = 'crypto_marketcap', style = {'width':'900px', 'height':'680px'}),
            dcc.Graph(id = 'crypto_correlation', style={'width':'900px', 'height':'680px', 'margin':'auto'})
        ])
    ])
])

])

# ------------------------------------------ PART 1 - INDEX BACKEND ---------------------------------------

# DCC Input that downloads the index data and keeps it for use by graph
@app.callback(Output('indice_graph_data', 'data'), [Input('indice_selector', 'value'),
                                                   Input('indice_date_selector','start_date'),
                                                   Input('indice_date_selector','end_date')])
def update_indice_data(indice_name, indice_start_date, indice_end_date):
    indice_name = get_index_symbol(indice_name)
    data = web.DataReader(indice_name,'yahoo', indice_start_date, indice_end_date).reset_index()
    return data.to_json(date_format = 'iso', orient = 'records')

# Index Graph showing multiple indexes based on user input from indice selector (id = 'indice_selector') and date picker (id = 'indice_date_selector')
@app.callback(Output('indice_graph', 'figure'), [Input('indice_selector','value'), 
                                                 Input('indice_date_selector','start_date'),
                                                 Input('indice_date_selector','end_date'),
                                                 Input('index_radio_items', 'value'),
                                                 Input('indice_graph_data','data')])
def update_index_line_graph(indice_name, indice_start_date, indice_end_date, radio_item, json_data):
    full_index_name = indice_name
    indice_name = get_index_symbol(indice_name)
    data = data_to_pandas(json_data)
    fig = go.Figure()
    for index in indice_name:
        fig.add_scatter(x = data[str(('Date',''))], y = data[str((radio_item,index))], mode = 'lines', name = full_index_name[indice_name.index(index)])
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )

    )
    return fig

# Correlation Graph based on user input from indice selector (id = 'indice_selector') and date picker (id = 'indice_date_selector')
@app.callback(Output('indice_correlation', 'figure'), [Input('indice_selector', 'value'), 
                                                       Input('indice_date_selector','start_date'),
                                                       Input('indice_date_selector','end_date'),
                                                       Input('index_radio_items', 'value'),
                                                       Input('indice_graph_data','data')])
def update_indice_corr_figure(indice_name, indice_start_date, indice_end_date, radio_item, json_data):
    full_index_name = indice_name
    indice_name = get_index_symbol(indice_name)
    corr_data = correlation_data(json_data, indice_name, radio_item)
    # data = web.DataReader(indice_name,'yahoo', indice_start_date, indice_end_date).reset_index()
    fig = px.imshow(corr_data,labels = dict(x = 'Index', y = 'Index', color = 'Correlation'), x = full_index_name, y = full_index_name, title = 'Correlation Graph of {} for {}'.format(
                                                                                                                                                        radio_dict.get(radio_item),",".join(full_index_name)))
    fig.update_layout(title_x=0.5)
    return fig

# ------------------------------------------ PART 2 - STOCK PRICE MOVEMENT BACKEND ---------------------------------------

# DCC Input that downloads the stock data and keeps it for use for graphs
@app.callback(Output('stock_graph_data', 'data'), [Input('stock_button', 'n_clicks'),
                                                  Input('stock_date_selector','start_date'),
                                                  Input('stock_date_selector','end_date')],
                                                [State('ticker_input','value')]) # The state is so that the graph does not update until user clicks on submit button
def update_stock_data(n_clicks, stock_start_date, stock_end_date, ticker_input):
    ticker_input = ticker_input.split(",")
    data = web.DataReader(ticker_input,'yahoo', stock_start_date, stock_end_date).reset_index()
    return data.to_json(date_format = 'iso', orient = 'records')

# Stock Graph showing multiple stocks based on user input from stock selector (id = 'stock_selector') and date picker (id = 'stock_date_selector')
@app.callback(Output('stock_graph', 'figure'), [Input('stock_button','n_clicks'), 
                                                 Input('stock_date_selector','start_date'),
                                                 Input('stock_date_selector','end_date'),
                                                 Input('stock_radio_items', 'value'),
                                                 Input('stock_graph_data','data'),
                                                 Input('ticker_input','value')])
def update_stock_line_graph(n_clicks, stock_start_date, stock_end_date, radio_item, json_data, ticker_input):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    fig = go.Figure()
    for ticker in ticker_input:
        fig.add_scatter(x = data[str(('Date',''))], y = data[str((radio_item,ticker))], mode = 'lines', name = ticker)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )

    )
    return fig

# Correlation Graph based on user input form ticker selector (id = 'ticker_input'), date picker (id = 'stock_date_selector') and update button (id = 'stock_button')
@app.callback(Output('stock_correlation', 'figure'), [Input('ticker_input', 'value'), 
                                                       Input('stock_date_selector','start_date'),
                                                       Input('stock_date_selector','end_date'),
                                                       Input('stock_radio_items', 'value'),
                                                       Input('stock_graph_data','data')])
def update_stock_corr_figure(stock_input, stock_start_date, stock_end_date, radio_item, json_data):
    stock_input = stock_input.split(",")
    corr_data = correlation_data(json_data, stock_input, radio_item)
    fig = px.imshow(corr_data,labels = dict(x = 'Ticker', y = 'Ticker', color = 'Correlation'), x = stock_input, y = stock_input, title = 'Correlation Graph of {} for {}'.format(radio_dict.get(radio_item),stock_input))
    fig.update_layout(title_x=0.5)
    return fig
    
# ------------------------------------------ PART 3 - STOCK FINANCIAL PERFORMANCE BACKEND ---------------------------------------

# DCC Input that downloads the stock analytics data and keeps it for use for the 4 graphs
@app.callback(Output('stock_analytics_data', 'data'), [Input('stock_button', 'n_clicks')],
                                                      [State('ticker_input','value')]) # The state is so that the graph does not update until user clicks on submit button
def update_stock_data(n_clicks, ticker_input):
    # Get the stock analytics from marketwatch
    stock_analytics_data = obtain_stock_analytics_data(ticker_input)
    # Create empty dataframe to obtain the stock analytics for user
    df = pd.DataFrame()
    for i in [4,6,7,8]:
        if i ==4:
            temp_df = stock_analytics(stock_analytics_data,ticker_input,i)
            temp_df['Category'] = stock_analytics_dict.get(i)
            df = temp_df.copy()
        else:
            temp_df = stock_analytics(stock_analytics_data, ticker_input, i)
            temp_df['Category'] = stock_analytics_dict.get(i)
            df = df.append(temp_df, ignore_index = True)    
    return df.to_json(date_format = 'iso', orient = 'records')

# Create graph for Stock analytics for Valuation Data
@app.callback(Output('stock_valuation', 'figure'), [Input('ticker_input','value'),
                                                    Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Valuation']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Valuation Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Create graph for Stock analytics for Liquidity Data
@app.callback(Output('stock_liquidity', 'figure'), [Input('ticker_input','value'),
                                                    Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Liquidity']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Liquidity Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Create graph for Stock analytics for Profitability Data
@app.callback(Output('stock_profitability', 'figure'), [Input('ticker_input','value'),
                                                    Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Profitability']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Profitability Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Create graph for Stock analytics for Capitalization Data
@app.callback(Output('stock_capitalization', 'figure'), [Input('ticker_input','value'),
                                                        Input('stock_analytics_data','data')])
def update_index_line_graph(ticker_input, json_data):
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    # Take the data for Valuation category only
    data = data[data['Category'] == 'Capitalization']
    graph_data = []
    for ticker in ticker_input:
        graph_data.append(go.Bar(x =  data['Description'], y = data[ticker], name = ticker))
    graph_layout = go.Layout(title = 'Stock Capitalization Bar Chart for {}'.format(",".join(ticker_input)), yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }
    
# ------------------------------------------ PART 4 - ARTICLES BACKEND ---------------------------------------

# DCC Input that downloads the stock news data for use from sentiment analyzer    
@app.callback(Output('articles_data', 'data'), [Input('stock_button', 'n_clicks'),
                                                Input('stock_date_selector', 'start_date'),
                                                Input('stock_date_selector','end_date')],
                                                [State('ticker_input','value')])
def update_sentiment_data(n_clicks,ticker_start_date, ticker_end_date, ticker_input):
    df1 = get_news_articles(ticker_input, ticker_start_date, ticker_end_date)
    df2 = news_articles_cleanup(df1)
    df3 = sentiment_scores_df(df2)
    return df3.to_json(date_format = 'iso', orient = 'records')

# Line Graph Input that visualizes the sentiment analysis data based on 
@app.callback(Output('line_graph_sentiment_graph', 'figure'), [Input('articles_data','data'),
                                                               Input('sentiment_radio_items', 'value')])
def update_sentiment_line_graph(json_data, sentiment_radio_item):
    data = data_to_pandas(json_data)
    ticker_input = list(data['Ticker'].unique())
    data.sort_values('Date').reset_index(drop = True)
    data = data[::-1]
    graph_data = []
    for ticker in ticker_input:
            graph_data.append(go.Scatter(x =  data['Date'], y = data[data['Ticker'] == ticker][sentiment_radio_item], mode = 'markers', name = ticker))
    graph_layout = go.Layout(title = '{} Sentiment Scatter Chart for {}'.format(sentiment_radio_item,",".join(ticker_input)), xaxis_title = 'Date', yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# Bar Graph Input that visualizes the sentiment analysis data based on ticker
@app.callback(Output('bar_graph_sentiment_graph', 'figure'), [Input('articles_data','data')])
def update_sentiment_line_graph(json_data):
    data = data_to_pandas(json_data)
    ticker_input = list(data['Ticker'].unique())
    graph_data = []
    for ticker in ticker_input:
        mydict = {}
        mydict['Ticker'] = ticker
        mydict['Negative'] = data[data['Ticker'] == ticker]['Negative'].mean()
        mydict['Positive'] = data[data['Ticker'] == ticker]['Positive'].mean()
        mydict['Neutral'] = data[data['Ticker'] == ticker]['Neutral'].mean()
        mydict['Compound'] = data[data['Ticker'] == ticker]['Compound'].mean()
        graph_data.append(mydict)
    
    data = pd.DataFrame(graph_data)
    columns_list = list(data.columns[1:])
    
    for ticker in ticker_input:
        for j in range(len(columns_list)):
            graph_data.append(go.Bar(x =  data[data['Ticker'] == ticker]['Ticker'], y = data[data['Ticker'] == ticker][columns_list[j]], name = columns_list[j]))
    graph_layout = go.Layout(title = 'Sentiment Bar Chart for {}'.format(",".join(ticker_input)), xaxis_title = 'Ticker', yaxis_title = "Value") 
    
    return{
        'data':graph_data,
        'layout':graph_layout
    }

# ------------------------------------------ PART 5 - CRYPTO ANALYSIS UI ---------------------------------------

# updates top 5 crypto data
@app.callback(Output('top5crypto_data','data'),[Input('crypto_refresh_button','n_clicks'), 
                                                State('crypto_ticker_input','value')])
def update_top5crypto_data(n_clicks, crypto_tickers,):
    tickers = crypto_tickers.split(',') + crypto_symbols
    top5crypto = web.DataReader(tickers, 'yahoo','2020-01-01').reset_index()
    top5crypto.fillna(0, inplace = True)
    return top5crypto.to_json(date_format = 'iso', orient = 'records')

# Updates top 25 graph data
@app.callback(Output('top25graph_data', 'data'), Output('crypto_refresh_button','n_clicks'),
                                                                                          [Input('crypto_refresh_button', 'n_clicks')])
def update_top25crypto_data(n_clicks):
    if n_clicks > 0:
        # Gets the top 25 crypto data
        df = top25crypto()
        n_clicks = 0
        return df.to_json(date_format = 'iso', orient = 'records'), n_clicks

# Updates the crypto market capitalization tree map
@app.callback(Output('crypto_marketcap', 'figure'), [Input('top25graph_data','data')])
def update_marketcap_graph(json_data):
    json_data = data_to_pandas(json_data)
    fig = px.treemap(json_data, path=['Name'], values='Market Cap', labels = 'Symbol')
    fig.update_layout(title = 'Market Capitalization of Top 25 Cryptocurrencies in Billions of USD', title_x = 0.5)
    return fig

# Updates teh line chart for crypto. Uses input from top 5 crypto data
@app.callback(Output('crypto_linegraph', 'figure'), [Input('crypto_radio_items','value'), 
                                                     Input('top5crypto_data','data'), 
                                                     Input('crypto_ticker_input','value')])
def update_top5crypto_grpah(radio_item, json_data, crypto_tickers):
    json_data = data_to_pandas(json_data)
    fig = go.Figure()
    tickers = crypto_tickers.split(',') + crypto_symbols
    for crypto in tickers:
        fig.add_scatter(x = json_data[str(('Date',''))], y = json_data[str((radio_item,crypto))], mode='lines', name = crypto)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    return fig

# Correlation Graph based on user input form ticker selector
@app.callback(Output('crypto_correlation', 'figure'), [Input('top5crypto_data', 'data'), 
                                                       Input('crypto_radio_items','value'),
                                                       Input('crypto_ticker_input','value')])
def update_stock_corr_figure(json_data, radio_item, crypto_tickers):
    tickers = crypto_tickers.split(',') + crypto_symbols
    corr_data = correlation_data(json_data, tickers, radio_item)
    fig = px.imshow(corr_data, labels = dict(x = 'Cryptocurrency', y = 'Cryptocurrency', color = 'Correlation'), x = tickers, y = tickers, 
                    title = 'Correlation Graph of {} for {}'.format(radio_dict.get(radio_item),tickers))
    fig.update_layout(title_x=0.5)
    return fig

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2022 16:10:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:10:56] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

127.0.0.1 - - [21/Jan/2022 16:10:56] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

...

127.0.0.1 - - [21/Jan/2022 16:19:36] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [21/Jan/2022 16:19:36] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

127.0.0.1 - - [21/Jan/2022 16:20:25] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Spyder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Spyder\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Spyder\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "D:\Spyder\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "D:\Spyder\lib\site-packages\dash\_callback.py", line 

127.0.0.1 - - [21/Jan/2022 16:20:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [21/Jan/2022 16:20:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:20:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:20:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:20:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 16:20:32] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
    graph_data = []
    ticker_input = ticker_input.split(",")
    data = data_to_pandas(json_data)
    for ticker in ticker_input:
        graph_data.append(go.Scatter(x =  data[str(('Date',''))], y = data[str((radio_item,ticker))], mode = 'lines', name = ticker))
    graph_layout = go.Layout(title = '{} Line Chart for {}'.format(radio_dict.get(radio_item),",".join(ticker_input)), xaxis_title = 'Date', yaxis_title = "Value")
    return{
        'data':graph_data,
        'layout':graph_layout
    }
